### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8436,2023-12-02,Brasil Nbb,17:00,Cerrado,São José,2.91,1.37,154.5,1.86,1.80,4.5,2.09,1.66,IeoQxYxl,0.343643,0.729927,0.537634,0.555556,0.073570,0.4,0.0,NaN,NaN,324.500,356.122135,1.097449,171.538,81.384377,0.474439,162.506,0.000,108.50,152.10,0.0,0.0,0.0,0.0,0.508853,0.023184,0.162163,0.66,0.132,14.469697,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8437,2023-12-02,Brasil Nbb,17:00,Flamengo,Franca,1.75,1.98,161.5,1.86,1.80,-2.5,2.02,1.71,WbkUyhhf,0.571429,0.505051,0.537634,0.555556,0.076479,0.0,0.0,NaN,NaN,96.104,12.087354,0.125774,112.892,33.224372,0.294302,0.000,0.000,109.47,171.72,0.0,0.0,0.0,0.0,0.087204,0.023184,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8438,2023-12-02,Brasil Nbb,18:00,R10 Score Vasco da Gama,Bauru,1.70,2.07,149.5,1.85,1.83,-3.5,2.09,1.66,ENvZzC70,0.588235,0.483092,0.540541,0.546448,0.071327,0.0,0.6,NaN,NaN,NaN,NaN,NaN,176.948,88.142539,0.498127,0.000,180.936,99.54,100.80,0.0,0.0,0.0,0.0,0.138795,0.007686,0.162163,0.00,0.000,inf,0.0,0.0,0.0,-5.0,-1.0,-1.07,0.0,0.0,0.0
8439,2023-12-02,Espanha Acb,14:00,Unicaja,Manresa,1.13,6.20,168.5,1.80,1.99,-12.5,2.03,1.75,b9ZLA6rq,0.884956,0.161290,0.555556,0.502513,0.046246,0.0,0.0,NaN,NaN,199.258,151.813419,0.761894,207.676,94.503618,0.455053,0.000,0.000,98.31,322.50,0.0,0.0,0.0,0.0,0.978180,0.070897,0.104757,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8440,2023-12-02,Espanha Acb,14:00,Zaragoza,Murcia,2.71,1.47,162.5,1.81,1.93,3.5,2.02,1.77,CCwQ9Qck,0.369004,0.680272,0.552486,0.518135,0.049276,0.0,0.0,NaN,NaN,187.128,46.804016,0.250118,144.846,46.172356,0.318769,0.000,0.000,182.36,119.32,0.0,0.0,0.0,0.0,0.419527,0.045376,0.093286,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768,2023-12-02,Israel Superliga,14:20,Hapoel Holon,H. Afula,1.18,4.64,167.5,1.86,1.87,-12.5,2.06,1.66,2TlH3rWj,0.847458,0.215517,0.537634,0.534759,0.062975,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,264.60,0.0,0.0,0.0,0.0,0.840752,0.003791,0.152066,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8769,2023-12-02,Macedônia Do Norte Prva Liga,13:00,FMP Akademija,Pelister,7.85,1.04,169.5,1.80,1.86,16.5,2.00,1.72,r9XduVzr,0.127389,0.961538,0.555556,0.537634,0.088927,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,533.82,109.60,1.0,0.0,0.0,0.0,1.083329,0.023184,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8770,2023-12-02,Noruega Blno,13:00,Gimle,Ammerud,1.07,7.50,159.5,1.80,1.86,-19.5,2.08,1.67,OEisMivL,0.934579,0.133333,0.555556,0.537634,0.067913,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,480.75,0.00,1.0,0.0,0.0,0.0,1.061073,0.023184,0.154621,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8771,2023-12-02,Portugal Lpb,16:00,Benfica,Lusitania,1.04,9.85,163.5,1.84,1.86,-28.5,2.00,1.72,plOT0bBF,0.961538,0.101523,0.543478,0.537634,0.063061,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,88.20,248.32,0.0,0.0,0.0,0.0,1.144097,0.007644,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
7,18:00,Eua Nba,Los Angeles Clippers,Golden State Warriors,227.5,1.80,1.0000,Over
8,19:00,Eua Nba,Charlotte Hornets,Minnesota Timberwolves,219.5,1.82,0.9999,Over
9,21:00,Eua Nba,Brooklyn Nets,Orlando Magic,223.5,1.80,1.0000,Over
10,21:00,Eua Nba,Detroit Pistons,Cleveland Cavaliers,222.5,1.83,0.9999,Over
11,22:00,Eua Nba,Chicago Bulls,New Orleans Pelicans,218.5,1.80,1.0000,Over
12,22:00,Eua Nba,Miami Heat,Indiana Pacers,239.5,1.88,1.0000,Over
13,22:00,Eua Nba,Milwaukee Bucks,Atlanta Hawks,248.5,1.80,1.0000,Over
86,15:00,Eua Ncaa,VMI,Presbyterian,148.5,1.91,0.8519,Over
90,16:00,Eua Ncaa,Buffalo,St. Bonaventure,139.5,1.91,0.8786,Over
95,16:00,Eua Ncaa,N. Carolina A&T,Citadel,148.5,1.91,0.9957,Over
